In [81]:
import pandas as pd

# def print_csv(file): # helper function to debug
#     with open(file, 'r') as csvfile:
#         csv_reader = csv.reader(csvfile)
#         for row in csv_reader:
#             print(row)

In [82]:
# test_names = [
#     "Jesus", "Paul", "David", "Moses", "Peter",
#     "Judah", "Saul", "Samson", "Elijah", "Jezebel"
# ]

character_names = [
    "Aaron", "Abel", "Abigail", "Abraham", "Absalom", "Achan", "Adam", "Adonijah", 
    "Amos", "Andrew", "Anna", "Apollos", "Aquila", "Asa", "Asher", "Barnabas", 
    "Bartholomew", "Boaz", "Cain", "Cornelius", "Cyrus", "Daniel", "David", "Deborah", 
    "Elijah", "Elisha", "Elizabeth", "Esau", "Esther", "Eve", "Ezekiel", "Felix", 
    "Gideon", "Hannah", "Hiram", "Hosea", "Huldah", "Isaac", "Isaiah", "Jacob", 
    "James", "Japheth", "Jared", "Jairus", "Jeremaih", "Jesus", "Job", "Joel", 
    "John", "John the Baptist", "Jonah", "Jonathan", "Joseph", "Joshua", "Josiah", 
    "Judah", "Jude", "Junia", "Lazarus", "Levi", "Lot", "Luke", "Lydia", "Manasseh", 
    "Martha", "Mary", "Mary Magdalene", "Matthias", "Melchizedek", "Micah", "Miriam", 
    "Mordecai", "Moses", "Nahum", "Nathan", "Nathanael", "Nehemiah", "Nicodemus", 
    "Noah", "Obadiah", "Paul", "Peter", "Philip", "Priscilla", "Rachel", "Rahab", 
    "Rhoda", "Ruth", "Samson", "Samuel", "Sarah", "Saul", "Seth", "Shem", "Silas", 
    "Simon", "Solomon", "Thomas", "Zechariah"
]

bible_paths = {
    "asv": "../../model_annotation/asv_out.csv",
    "fbv": "../../model_annotation/fbv_out.csv",
    "web": "../../model_annotation/web_out.csv",
    "wmb": "../../model_annotation/wmb_out.csv",
    "kjv": "../../model_annotation/kjv_out.csv"
}

def mentions_character(text):
    for name in character_names:
        if name in text:
            return True
    return False

def most_frequent_sentiment(sentiments):
    max_sentiment = max(sentiments, key=sentiments.get)
    max_count = sentiments[max_sentiment]
    counts = list(sentiments.values())
    if counts.count(max_count) > 1:
        return None
    return max_sentiment

In [83]:
# # uncomment this block and next for sentiment only

# result_data = {name: {} for name in character_names}

# # finding most frequent sentiment for each character in each translation
# for bible, path in bible_paths.items():
#     df = pd.read_csv(path)
#     bible_sentiment_count = {name: {1: 0, 2: 0, 3: 0} for name in character_names}
#     for index, row in df.iterrows():
#         sentiment = int(row['sentiment'])
#         if sentiment in [1, 2, 3]: # disregard the sentiments labeled as 0 from model annotations
#             for name in character_names:
#                 if name in row['text']:
#                     bible_sentiment_count[name][sentiment] += 1
#     for name in character_names:
#         max_sentiment = most_frequent_sentiment(bible_sentiment_count[name])
#         if max_sentiment is not None:
#             result_data[name][bible] = max_sentiment

# # creating table for visualization
# result_df = pd.DataFrame(result_data).transpose()
# result_df.columns = ['ASV', 'FBV', 'WEB', 'WMB', 'KJV']
# result_df = result_df.replace({1: 'Neutral (1)', 2: 'Positive (2)', 3: 'Negative (3)'}, inplace=False)
# result_df.fillna('N/A', inplace=True) # replace NaN with N/A

# # exporting to csv to process further
# result_df.to_csv('char_analysis.csv')
# result_df

In [84]:
# df = pd.read_csv('char_analysis.csv', index_col=0)
# conflict_characters = []

# for index, row in df.iterrows():
#     if len(row.dropna().unique()) > 1:
#         conflict_characters.append(index)

# conflict_df = df.loc[conflict_characters]
# conflict_df.fillna('N/A', inplace=True)
# conflict_df.to_csv('conflict_sentiments.csv')
# conflict_df

In [85]:
# uncomment for sentiment and average confidence scores

result_data = {name: {} for name in character_names}

# finding most frequent sentiment AND average confidence for that sentiment for each character in each translation
for bible, path in bible_paths.items():
    df = pd.read_csv(path)
    bible_sentiment_count = {name: {1: [], 2: [], 3: []} for name in character_names}
    for index, row in df.iterrows():
        sentiment = int(row['sentiment'])
        confidence = float(row['confidence'])
        if sentiment in [1, 2, 3]:  # disregard the sentiments labeled as 0 from model annotations (runtime error with web.csv)
            for name in character_names:
                if name in row['text']:
                    bible_sentiment_count[name][sentiment].append(confidence)
    for name in character_names:
        max_sentiment = most_frequent_sentiment(bible_sentiment_count[name])
        if max_sentiment is not None:
            result_data[name][bible + ' sent'] = max_sentiment
            result_data[name][bible + ' conf'] = sum(bible_sentiment_count[name][max_sentiment]) / len(
                bible_sentiment_count[name][max_sentiment])

# creating table for visualization
conf_df = pd.DataFrame(result_data).transpose()
conf_df = conf_df.replace({1: 'Neutral (1)', 2: 'Positive (2)', 3: 'Negative (3)'}, inplace=False)
conf_df.fillna('N/A', inplace=True)  # replace NaN with N/A

# exporting to csv to process further
conf_df.to_csv('char_analysis_confidence.csv')
conf_df

C:\Users\nicho\AppData\Local\Temp\ipykernel_12244\2623781647.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  conf_df.fillna('N/A', inplace=True)  # replace NaN with N/A


,asv sent,asv conf,fbv sent,fbv conf,web sent,web conf,wmb sent,wmb conf,kjv sent,kjv conf
Aaron,Positive (2),0.656867,Neutral (1),0.771687,Neutral (1),0.750036,Neutral (1),0.750695,Neutral (1),0.745465
Abel,Neutral (1),0.729914,Positive (2),0.77766,Neutral (1),0.751695,Neutral (1),0.706254,Neutral (1),0.755795
Abigail,Neutral (1),0.744494,Neutral (1),0.751019,Positive (2),0.879762,Positive (2),0.78165,Positive (2),0.739775
Abraham,Positive (2),0.671157,Negative (3),0.629896,Positive (2),0.727317,Positive (2),0.716407,Positive (2),0.68795
Absalom,Neutral (1),0.728173,Positive (2),0.67598,Neutral (1),0.724525,Neutral (1),0.720686,Neutral (1),0.715368
...,...,...,...,...,...,...,...,...,...,...
Silas,Neutral (1),0.734062,Positive (2),0.94041,Neutral (1),0.765707,Neutral (1),0.756802,Neutral (1),0.703835
Simon,Neutral (1),0.73474,Positive (2),0.726079,Positive (2),0.768289,Neutral (1),0.737162,Neutral (1),0.724411
Solomon,Neutral (1),0.733284,Neutral (1),0.770792,Neutral (1),0.761866,Neutral (1),0.762615,Neutral (1),0.74307
Thomas,Positive (2),0.559892,Neutral (1),0.729165,Neutral (1),0.819111,Neutral (1),0.730789,Neutral (1),0.71523


In [86]:
df = pd.read_csv('char_analysis_confidence.csv', index_col=0)
df = df.dropna().replace('N/A', pd.NA)
conflict_characters = []

# check for if character has been labeled as neu, pos AND neg in one or more translations
for index, row in df.iterrows():
    unique_sentiments = row.nunique()
    if unique_sentiments >= 3:
        conflict_characters.append((index, unique_sentiments))

# grab top 10 based on variance in confidence scores
conflict_characters.sort(key=lambda x: x[1], reverse=True)
top10 = conflict_characters[:10]

conflicting_df = df.loc[[character[0] for character in top10]]
conflicting_df.to_csv('top10conflict_confidence.csv')
conflicting_df

,asv sent,asv conf,fbv sent,fbv conf,web sent,web conf,wmb sent,wmb conf,kjv sent,kjv conf
Adam,Neutral (1),0.764069,Negative (3),0.741863,Positive (2),0.819912,Negative (3),0.595998,Neutral (1),0.742103
Esau,Negative (3),0.677719,Neutral (1),0.777297,Positive (2),0.710058,Positive (2),0.714411,Negative (3),0.653098
Esther,Neutral (1),0.736052,Positive (2),0.663951,Neutral (1),0.768367,Negative (3),0.547551,Neutral (1),0.734712
Eve,Neutral (1),0.652076,Negative (3),0.764182,Negative (3),0.682871,Negative (3),0.674092,Positive (2),0.688283
Isaiah,Neutral (1),0.722235,Positive (2),0.764847,Negative (3),0.773360,Neutral (1),0.683271,Neutral (1),0.732276
Judah,Neutral (1),0.733739,Positive (2),0.717769,Negative (3),0.670462,Negative (3),0.658285,Neutral (1),0.728644
Lazarus,Neutral (1),0.714507,Negative (3),0.686164,Negative (3),0.758334,Positive (2),0.750052,Negative (3),0.557204
Miriam,Neutral (1),0.722967,Negative (3),0.786314,Positive (2),0.565443,Positive (2),0.754526,Positive (2),0.535464
Peter,Neutral (1),0.724145,Negative (3),0.658112,Positive (2),0.709006,Positive (2),0.706987,Neutral (1),0.726222
Rahab,Neutral (1),0.708860,Neutral (1),0.737561,Positive (2),0.928969,Negative (3),0.644632,Positive (2),0.921113
